In [18]:
import requests
from typing import Tuple, List
from math import log
import pandas as pd


In [19]:
currencies = ('PLN', 'EUR', 'USD', 'RUB', 'INR', 'MXN')
api_key = 'fe69bd3037f74f699cf2bf8070b44374'

def get_rates(currencies: Tuple[str, ...], api_key: str) -> List[List[float]]:
    rates = []
    response = requests.get(f'https://openexchangerates.org/api/latest.json?app_id={api_key}')
    data = response.json()['rates']
    rates_to_usd = {currency: data.get(currency, 0) for currency in currencies}

    for currency in currencies:
        rates.append([rates_to_usd[currency] / rates_to_usd[curr] if rates_to_usd[curr] != 0 else 0 for curr in currencies])
    return rates

def negate_logarithm_convertor(graph: Tuple[Tuple[float]]) -> List[List[float]]:
    ''' log of each rate in graph and negate it'''
    result = [[-log(edge) if edge != 0 else float('inf') for edge in row] for row in graph]
    return result

def arbitrage(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    ''' Calculates arbitrage situations and prints out the details of this calculations'''

    trans_graph = negate_logarithm_convertor(rates_matrix)

    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n

    for source in range(n):
        min_dist[source] = 0

        for _ in range(n-1):
            for source_curr in range(n):
                for dest_curr in range(n):
                    if source_curr == dest_curr or trans_graph[source_curr][dest_curr] == float('inf'):
                        continue
                    if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                        min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                        pre[dest_curr] = source_curr

        for source_curr in range(n):
            for dest_curr in range(n):
                if source_curr == dest_curr or trans_graph[source_curr][dest_curr] == float('inf'):
                    continue
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    print_cycle = [dest_curr, source_curr]
                    while pre[source_curr] not in print_cycle:
                        source_curr = pre[source_curr]
                        print_cycle.append(source_curr)
                    print_cycle.append(pre[source_curr])
                    if print_cycle[0] == print_cycle[-1]:
                        print(f"Arbitrage Opportunity: \n{' --> '.join([currencies[p] for p in print_cycle[::-1]])}")

In [20]:
rates = get_rates(currencies, api_key)
df_rates = pd.DataFrame(rates, columns=currencies, index=currencies)
df_rates

,PLN,EUR,USD,RUB,INR,MXN
PLN,1.000000,4.299777,3.946774,0.040625,0.046969,0.199964
EUR,0.232570,1.000000,0.917902,0.009448,0.010924,0.046506
USD,0.253371,1.089441,1.000000,0.010293,0.011901,0.050665
RUB,24.615367,105.840593,97.151292,1.000000,1.156152,4.922190
INR,21.290772,91.545572,84.029864,0.864938,1.000000,4.257390
MXN,5.000897,21.502744,19.737412,0.203162,0.234886,1.000000


In [21]:
arbitrage(currencies, rates)

Arbitrage Opportunity: 
RUB --> MXN --> INR --> EUR --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> USD --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> EUR --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> USD --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> PLN --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> EUR --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> USD --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> PLN --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> EUR --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> USD --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> PLN --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> EUR --> RUB
Arbitrage Opportunity: 
RUB --> MXN --> INR --> USD --> RUB
